In [7]:
!pip install langchain chromadb tiktoken transformers sentence_transformers pypdf

In [8]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [9]:
#from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf')
pages = loader.load_and_split()

In [10]:
# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=tiktoken_len
)

docs = text_splitter.split_documents(pages)

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}
hf = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [12]:
db = Chroma.from_documents(docs, hf)

In [13]:
# query it
query = '4k recording'
docs = db.similarity_search(query)

print(docs[0].page_content)

Hyperlapse 4K/2.7K/1080p@25/30fps: Auto/×2/×5/×10/×15/×30
Timelapse 4K/2.7K/1080p@25/30fps
Intervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s
Duration: 5/10/20/30 mins, 1/2/3/5/∞ hours
Motionlapse 4K/2.7K/1080p@25/30fps
Intervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s
Duration: 5/10/20/30 mins, 1/2/3/5 hours
Supports setting four positions
Low-Light Video 4K (16:9): 3840×2160@24/25/30fps
1080p: 1920×1080@24/25/30fps
Max Video Bitrate 130 Mbps
Supported File System exFAT
Photo Format JPEG/RAW
Video Format MP4 (H.264/HEVC)
Built-in Storage 
CapacityThe camera does not have built-in storage, but storage capacity can 
be expanded by inserting a microSD card.
Audio Output 48 kHz 16-bit; AAC
Battery
Type LiPo
Capacity 1300 mAh
Energy 10.01 Wh


In [14]:
# save to disk
db2 = Chroma.from_documents(docs, hf , persist_directory='./chroma_db')
docs = db2.similarity_search(query)

In [15]:
db3 = Chroma(persist_directory='./chroma_db', embedding_function=hf)
docs= db3.similarity_search(query)

print(docs[0].page_content)

Hyperlapse 4K/2.7K/1080p@25/30fps: Auto/×2/×5/×10/×15/×30
Timelapse 4K/2.7K/1080p@25/30fps
Intervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s
Duration: 5/10/20/30 mins, 1/2/3/5/∞ hours
Motionlapse 4K/2.7K/1080p@25/30fps
Intervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s
Duration: 5/10/20/30 mins, 1/2/3/5 hours
Supports setting four positions
Low-Light Video 4K (16:9): 3840×2160@24/25/30fps
1080p: 1920×1080@24/25/30fps
Max Video Bitrate 130 Mbps
Supported File System exFAT
Photo Format JPEG/RAW
Video Format MP4 (H.264/HEVC)
Built-in Storage 
CapacityThe camera does not have built-in storage, but storage capacity can 
be expanded by inserting a microSD card.
Audio Output 48 kHz 16-bit; AAC
Battery
Type LiPo
Capacity 1300 mAh
Energy 10.01 Wh


In [16]:
docs = db3.similarity_search_with_relevance_scores(query, k=3)

print('가장 유사한 문서: \n\n {}\n\n'.format(docs[0][0].page_content))
print('문서 유사도:\n {}'.format(docs[0][1]))

가장 유사한 문서: 

 Hyperlapse 4K/2.7K/1080p@25/30fps: Auto/×2/×5/×10/×15/×30
Timelapse 4K/2.7K/1080p@25/30fps
Intervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s
Duration: 5/10/20/30 mins, 1/2/3/5/∞ hours
Motionlapse 4K/2.7K/1080p@25/30fps
Intervals: 0.5/1/2/3/4/5/6/8/10/15/20/25/30/40/60 s
Duration: 5/10/20/30 mins, 1/2/3/5 hours
Supports setting four positions
Low-Light Video 4K (16:9): 3840×2160@24/25/30fps
1080p: 1920×1080@24/25/30fps
Max Video Bitrate 130 Mbps
Supported File System exFAT
Photo Format JPEG/RAW
Video Format MP4 (H.264/HEVC)
Built-in Storage 
CapacityThe camera does not have built-in storage, but storage capacity can 
be expanded by inserting a microSD card.
Audio Output 48 kHz 16-bit; AAC
Battery
Type LiPo
Capacity 1300 mAh
Energy 10.01 Wh


문서 유사도:
 0.45507221416936583


# FAISS

In [17]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 40.7 MB/s eta 0:00:00


In [18]:
from langchain.vectorstores import FAISS

pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=tiktoken_len
)
docs = text_splitter.split_documents(pages)

model_name='jhgan/ko-sbert-nli'
model_kwargs={'device':'cpu'}
encode_kwargs={'normalize_embeddings':True}
ko=HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [19]:
db = FAISS.from_documents(docs, ko)

In [20]:
query = 'Who is DJI'
docs = db.similarity_search(query)
print(docs[0].page_content)


If you have any questions about this document, please 
contact DJI by sending a message to DocSupport@dji.com .
DJI and OSMO are trademarks of DJI. 
Copyright © 2023 DJI OSMO All Rights Reserved.WE ARE HERE FOR YOU
Contact 
DJI SUPPORT
https://www.dji.com/osmo-pocket-3/downloadsThis content is subject to change.


In [24]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores

[(Document(page_content='If you have any questions about this document, please \ncontact DJI by sending a message to DocSupport@dji.com .\nDJI and OSMO are trademarks of DJI. \nCopyright © 2023 DJI OSMO All Rights Reserved.WE ARE HERE FOR YOU\nContact \nDJI SUPPORT\nhttps://www.dji.com/osmo-pocket-3/downloadsThis content is subject to change.', metadata={'source': '/content/DJI_Osmo_Pocket_3_User_Manual_v1.0_en.pdf', 'page': 39}),
  0.81629145),
 (Document(page_content='DJI Osmo Pocket 3 User Manual© 2023 DJI All Rights Reserved. \u200327DJI Mic 2 Transmitter Status LEDs\n1 2\nDJI Mic 2 Transmitter Connection\nFollow the steps below to connect to DJI Mic 2 (hereinafter referred to as “transmitter”).\n1. Swipe down from the top of the screen and enter the control menu. Tap the settings icon \nand select Wireless Mic > TX1/TX2, and the camera is ready to link with a transmitter.\n2. Press and hold the power button of the transmitter for two seconds to power on.1 Recording Status LED\nBli

In [25]:
db.save_local('faiss_index')

In [26]:
new_db = FAISS.load_local('faiss_index', ko)

query = 'video edit'
docs = new_db.similarity_search_with_relevance_scores(query, k=3)

In [28]:
print('질문: {} \n'.format(query))
for i in range(len(docs)):
  print('{0}번째 유사 문서 유사도 \n{1}'.format(i+1, round(docs[i][1],2)))
  print('-' * 100)
  print(docs[i][0].page_content)
  print('\n')
  print(docs[i][0].metadata)
  print('-' * 100)

질문: video edit 

1번째 유사 문서 유사도 
0.49
----------------------------------------------------------------------------------------------------
DJI Osmo Pocket 3  User Manual26 © 2023 DJI All Rights Reserved. screen displays Webcam. Press the shutter/record button to start recording. Swipe left from the 
right edge of the screen to view parameters such as, Exposure, White Balance, and Focus Mode. 
Users can preview recorded videos after exiting the Webcam mode.
Microphone Connection
Osmo Pocket 3 can be linked to an external microphone, including DJI Mic, DJI Mic 2, or any 
third-party digital USB-C microphones that support 48K/16bit.
Connecting to DJI Mic 2
DJI Mic 2 Transmitter Overview
1
2
3
4 567
1.  Recording Status LED
 Indicates the record status of the transmitter.
2.  Data Port (USB-C)
 For charging and copying audio files or updating firmware after connecting to a computer. 
3.  Record Button
 Press once to start or stop recording in standalone recording mode.
4.  3.5mm TRS Input
 

In [40]:
query = 'Is an app provided?'
docs = new_db.max_marginal_relevance_search(query, k=3, fetck_k=10, lamda_mult=0.3)  # 다양성을 추구한다.
len(docs)

3

In [41]:
print('질문: {} \n'.format(query))
for i in range(len(docs)):
  print('{0}번째 유사 문서 유사도'.format(i+1))
  print('-' * 100)
  print(docs[i].page_content)
  print('\n')
  print(docs[i].metadata)
  print('-' * 100)

질문: Is an app provided? 

1번째 유사 문서 유사도
----------------------------------------------------------------------------------------------------
DJI Osmo Pocket 3 User Manual© 2023 DJI All Rights Reserved.  31Connecting to the DJI Mimo App
1. Power on Osmo Pocket 3.
2. Enable Wi-Fi and Bluetooth on the mobile device.
3. Launch DJI Mimo, tap  and follow the instructions to connect to Osmo Pocket 3. 
• When Osmo Pocket 3 is linked with DJI Mic 2 Transmitter and working in the 2.4GHz 
Wi-Fi frequency band, DJI Mimo cannot connect to Osmo Pocket 3. Either change the 
Wi-Fi frequency band of Osmo Pocket 3 to 5.8GHz or disconnect DJI Mic 2 Transmitter 
before connecting to the DJI Mimo app.
• If there is a problem when connecting to DJI Mimo, follow the steps below:
a. Make sure both Wi-Fi and Bluetooth are enabled on the mobile device.
b. Make sure DJI Mimo is updated to the latest firmware version.
c. Swipe down from the top of the screen to enter the control menu, tap the settings 
icon, and 